# IBMDS Capstone Proj: Segmenting and Clustering Neighborhoods in Toronto

## Part I: Data Improt and Preprocessing

## 1. Data Import by scraping the Wikipedia page

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


#### Scraping the web page

In [22]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wp=requests.get(url).text
wp_scrape=BeautifulSoup(wp, 'lxml')

#### Find the target table
webpage-> Inspect -> locate the table in HTML:   
 i) div class="mw-parser-output"  
 ii) table class="wikitable sortable jquery-tablesorter"  
          tab: 'table"   
          class: "wikitable sortable jquery-tablesorter"  

In [23]:
wp_table=wp_scrape.find('table', class_="wikitable sortable")

print("the data type is:",type(wp_table))
print("the name is:", wp_table.name)
print("the Table Header is:", wp_table.tr.text, wp_table.tr)

the data type is: <class 'bs4.element.Tag'>
the name is: table
the Table Header is: 
Postcode
Borough
Neighbourhood
 <tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


## 2. Read the data into a pandas Dataframe

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [24]:

col_name=['Postcode','Borough','Neighborhood']
wp_df=pd.DataFrame(columns=col_name)
wp_df

,Postcode,Borough,Neighborhood


In [25]:
for tr in wp_table.find_all('tr'):
    i=0
    tx=['','','']
    for td in tr.find_all('td'):
        tx[i]=td.text
        #print(i, tx[i])
        i=i+1
    #print(tx[0], tx[1], tx[2])
    wp_df=wp_df.append({'Postcode':tx[0],'Borough':tx[1], 'Neighborhood': tx[2].rstrip('\n')}, ignore_index=True)
print("DataFrame's shape:", wp_df.shape)
wp_df.head(10)


DataFrame's shape: (289, 3)


,Postcode,Borough,Neighborhood
0,,,
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


## 3. Data Preprocessing


#### 3-1) Filter out "Not Assigned":
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [26]:
wp_df['Borough'].value_counts()  

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
                     1
Name: Borough, dtype: int64

In [27]:
wp_df=wp_df[( wp_df['Borough']!='Not assigned') & (wp_df['Borough']!="" )]
wp_df['Borough'].value_counts()  

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

#### 3-2) Not Assigned neighborhood
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [28]:
wp_df.loc[wp_df['Neighborhood']=='Not assigned','Neighborhood']=wp_df['Borough']
wp_df.head(10)

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


#### 3-3) Combine the neighborhood for the same FSA:
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


In [30]:
wp_df=wp_df.groupby(['Postcode','Borough']).agg(','.join)
wp_df.head()
# reset_index to realign Postcode and Borough
wp_df_final=wp_df.reset_index()
wp_df_final.head(5)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### 3-4) show the shape of the DataFrame
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [31]:
wp_df_final.shape

(103, 3)